In [1]:
import numpy as np
import pandas as pd
import ast

# from sklearn.preprocessing import StandardScaler,Imputer

# from sklearn import metrics
# from scipy import spatial
import statistics


# from sklearn.linear_model import LinearRegression
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.metrics.pairwise import cosine_similarity

# import matplotlib.pyplot as plt
from itertools import islice



In [7]:
import math

In [8]:
ln=math.log

In [22]:
# Kaggle dataset
df=pd.read_csv("big_five_scores_modified.csv",delimiter=";")
df.pop("case_id")
df.columns=['Country', 'Age',"Sex","Agreeableness","Extraversion","Openness","Conscientiousness","Neuroticism","Skills","Looking for","Languages"]
to_pop=["Sex"]
for i in to_pop:
    df.pop(i)
df=df.drop(df.index[1000:])
df.head()
# # Seb's dataset
# fake_users=pd.read_csv("10_personalities.csv",delimiter=";")
# connections=pd.read_csv("connections_data.csv")
# print(fake_users.head())

fake_users=df
fake_users.head()

KeyError: 'case_id'

In [2]:
fake_users=pd.read_csv("../data/raw/BAZREVISEDPERSONAS.csv",delimiter=";")
# to_pop=["Predicted Match from OCEAN"]
# for i in to_pop:
#        fake_users.pop(i)
# fake_users.columns=["Openness","Conscientiousness","Extraversion","Agreeableness","Neuroticism","Skills","Looking for"]
fake_users.head()

,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Skills,Looking for,Language
0,25,25,75,75,25,Data Analysis,"Web Design, Back-End Development",English
1,75,25,85,85,25,"Back-End Development, Web design",Data Analysis,English
2,45,85,45,25,25,"Branding, Risk Assessment","Leadership, Communication, Powerpoint",French
3,75,35,65,25,25,"Coaching, Design, Web Design,","Mobile Development, UI/UX",Chinese
4,75,35,25,45,65,"Online Marketing, Design, UI/UX","Software Engineering, Back-end development, ac...",German


In [3]:
big5=["Openness","Conscientiousness","Extraversion","Agreeableness","Neuroticism"]


for i in big5:
    fake_users[i]*=0.01

In [4]:
fake_users["Neuroticism"]=1-fake_users["Neuroticism"]

In [5]:
fake_users["Neuroticism"]

0    0.75
1    0.75
2    0.75
3    0.75
4    0.35
5    0.25
6    0.85
7    0.50
8    0.25
9    0.85
Name: Neuroticism, dtype: float64

In [10]:

from itertools import islice


class Recommendation:
    def __init__(self,user_id,df):
        self.big5=["Extraversion","Agreeableness","Conscientiousness","Neuroticism","Openness"]
        self.importance_index={"Openness":0.7,"Extraversion":0.3,"Agreeableness":0.5,"Neuroticism":0.4,"Conscientiousness":0.6}
        self.reverse_threshold=50    
        self.df=df
#         self.connections=connections
        self.overall_scores={}
        self.user_id=user_id


    
    def get_recommendations_list(self,n):
        """
        input:the number of first elements
        outputs: sorted dictionary with similarity scores between target user and each other user
        """
        
        recommendation=self.main_recommender(self.user_id,self.overall_scores,self.df)
        sorted_recommendation=self.sorted_dictionary(recommendation)
        return self.take(n,sorted_recommendation.items())

    def take(self,n, iterable):
        "Return first n items of the iterable as a list"
        return list(islice(iterable, n))
        


    def sorted_dictionary(self,dictionary):
        """
        Returns sorted dictionary 
        """
        return {k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1],reverse=True)}
    def main_recommender(self,user_id,overall_scores,df):
        """
        input: user_id (integer), targeted_skills (list), personality_traits (dictionary)
        outputs: people suggestions as a dictionary with score
        """
    #   load the dataset 
        self.personality_language_age(df)
        self.skills_analyser(df)
#         self.count_mutual_connections()
        
        return self.overall_scores


    def personality_language_age(self,df):
        """
        input: df (pandas dataframe), personality traits (list)
        outputs: dictionary {user_index:mean_difference}
        """
#       Split the language
#         example_user_language=ast.literal_eval(fake_users.loc[self.user_id]["Languages"])
        example_user_language=fake_users.loc[self.user_id]["Language"]


#         example_user_age=fake_users.loc[self.user_id]["Age"]
#         example_user_country=fake_users.loc[self.user_id]["Country"]
#         common_language=False
            # Cosine similarity
        user_qualities={}
        # Filling our user qualities
        for i in self.big5:
            user_qualities[i]=self.df.loc[self.user_id][i]
        users_mean_difference=[]

        for index, row in df.iterrows():
            mean_difference=0
            if index!=self.user_id:
                
                
#             Create the list of the user languages
                current_user_language=row["Language"]
#                 current_user_language=ast.literal_eval(row["Languages"])

    
#                 current_user_country=row["Country"]
#                 current_user_age=row["Age"]
#                 Find the age difference
#                 age_difference=abs(example_user_age-current_user_age)
                for i in self.big5:
                    row_trait=row[i]
                    users_trait=user_qualities[i]
                    difference=0
#                 Keep track of counted rows
                    counted=False

# Check if the user quality is more than 40. Else, make a reverce relationship
#                     if user_qualities[i]<40 or row[i]<40:
                        
#                         difference+=abs(ln(row_trait/users_trait))*self.importance_index[i]
#                         mean_difference+=(1-difference)
#                         counted=True
                    if not counted:

                        mean_difference+=abs(ln(row_trait/users_trait))*self.importance_index[i]

                mean_difference/=len(self.big5)
                
#                 Store the final reverse mean difference (the bigger the better) in the dictionary
                self.overall_scores[index]=(1-mean_difference)
#                 if example_user_language!=current_user_language:
#                     self.overall_scores[index]*=0.7
                
#                 for i in example_user_language:

#                     if i in current_user_language:


#                         common_language=True

#                 if common_language==False:        
#                     self.overall_scores[index]=int(self.overall_scores[index])*0.7
                
# #                 Check the country
#                 if current_user_country!=example_user_country:
#                     self.overall_scores[index]*=0.8
                
        return self.overall_scores
    
    def skills_analyser(self,df):
#         example_user_skills=ast.literal_eval(fake_users.loc[self.user_id]["Skills"])
        example_user_skills=self.df.loc[self.user_id]["Skills"].split(",")
#         looking_for_skills=ast.literal_eval(fake_users.loc[self.user_id]["Looking for"])
        looking_for_skills=self.df.loc[self.user_id]["Looking for"].split(",")


        # get the targeted skills lost
        targeted_skills=[]
        # Delete the final comma 
        del example_user_skills[-1]
        user_skills_list=[]
        
        for index,row in fake_users.iterrows():
            
            if index!=self.user_id:
                current_user_skills=row["Skills"].split(",")
                current_user_looking_for=row["Looking for"].split(",")

#                 Delete the final comma
                del current_user_skills[-1]
#     Finding the similarities between two lists
                similarities_xy=set(looking_for_skills) & set(current_user_skills)
                similarities_yx=set(current_user_looking_for) & set(example_user_skills) 
#     Calculating a similarity percentage(score)
                similarity_xy=len(similarities_xy)/len(looking_for_skills)
                similarity_yx=len(similarities_yx)/len(current_user_looking_for)
                similarities=statistics.mean([similarity_xy,similarity_yx])

#                 similarities=set(example_user_skills) & set(curent_user_looking_for)
                user_skills_list.append([index,similarities])
                self.overall_scores[index]+=similarities
                


        return self.overall_scores

#     def count_mutual_connections(self):
#         """
#         input:user_id (integer), connections datafraem
#         output: dictionary {user_index:number_of mutual connections}
#         """


#         user_connections=[]
#         print(self.connections.head())


#         for index, row in self.connections.iterrows():
#             if row["connection_initiator"]==self.user_id and row["connection"]!=self.user_id:
#                 user_connections.append(row["connection"])
#             if row["connection"]==self.user_id and row["connection_initiator"]!=self.user_id:
#                 user_connections.append(row["connection_initiator"])
            
#         mutual_connections={}
#         for index, row in self.connections.iterrows():
#             for i in user_connections:
#                 if i==row["connection"] and i!=self.user_id and row["connection_initiator"]!=self.user_id:
#                     try:
#                         mutual_connections[row["connection_initiator"]]+=1
#                     except KeyError:
#                         mutual_connections[row["connection_initiator"]] =1
#                 if i==row["connection_initiator"] and i!=self.user_id and row["connection"]!=self.user_id:
#                     try:
#                         mutual_connections[row["connection"]]+=1
#                     except KeyError:
#                         mutual_connections[row["connection"]] = 1
# #         print(mutual_connections)
#         for key,value in mutual_connections.items():
#             # add all connections to the overall score
#             if key<len(self.df):
#                 self.overall_scores[key]+=value
#         return (self.overall_scores)


recommender=Recommendation(7,fake_users)
listrec=recommender.get_recommendations_list(9)
print("The best matches for user {} are:\n".format(recommender.user_id)+ str(recommender.get_recommendations_list(9)))

The best matches for user 7 are:
[(2, 0.8881298369060315), (9, 0.8645311425790803), (0, 0.8440084612896078), (3, 0.7787987964520333), (6, 0.7212515175246317), (1, 0.6701766380034315), (5, 0.6496653350663693), (4, 0.617483322707487), (8, 0.6082350942223472)]


In [21]:
print(fake_users.iloc[listrec[1][0]])
print(fake_users.iloc[recommender.user_id])
# print(df.loc[[60]])

Openness                                       0.75
Conscientiousness                              0.35
Extraversion                                   0.65
Agreeableness                                  0.25
Neuroticism                                    0.75
Skills               Coaching, Design, Web Design, 
Looking for               Mobile Development, UI/UX
Language                                    Chinese
Name: 3, dtype: object
Openness                                                         0.25
Conscientiousness                                                 0.7
Extraversion                                                     0.75
Agreeableness                                                    0.75
Neuroticism                                                       0.5
Skills               Leadership, Power point, Coaching, Communication
Looking for                     Risk Assessment, Web Design, Branding
Language                                                       French
N

In [118]:
df.head(listrec[0][0]+1).tail(1)

,Age,Agreeableness,Extraversion,Openness,Conscientiousness,Neuroticism,Skills,Looking for,Languages
70,45,0.763333,0.523333,0.76,0.806667,0.856667,"['AWS', 'Premiere Pro', 'Digital Marketing', '...","['Accounting', 'PR']","['French', 'English']"


In [119]:
df.head(recommender.user_id+1).tail(1)

,Age,Agreeableness,Extraversion,Openness,Conscientiousness,Neuroticism,Skills,Looking for,Languages
70,45,0.763333,0.523333,0.76,0.806667,0.856667,"['AWS', 'Premiere Pro', 'Digital Marketing', '...","['Accounting', 'PR']","['French', 'English']"
